# Stage 5: Database Integration & Dual-Axis Sentiment Ratios

1. Create `article_sentiment` table with **both axes** (labor stance + railroad outlook)
2. Calculate **within-newspaper sentiment ratios** for DiD analysis

**Output**:
- `article_sentiment` table in `newspapers.db`
- `data/labor_sentiment_ratios.csv` (newspaper-year level)
- `data/railroad_sentiment_ratios.csv` (newspaper-year level)

In [ ]:
import sqlite3
import json
import os
import logging
import pandas as pd
from pathlib import Path
from tqdm import tqdm

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler()]
)
log = logging.getLogger(__name__)

DB_PATH = os.path.join('..', 'data', 'newspapers.db')
RESULTS_FILE = Path('data/sentiment_results/all_sentiment_results.json')
LABOR_RATIOS_OUTPUT = Path('data/labor_sentiment_ratios.csv')
RAILROAD_RATIOS_OUTPUT = Path('data/railroad_sentiment_ratios.csv')

In [ ]:
def load_json(filepath):
    with open(filepath, encoding='utf-8') as f:
        return json.load(f)

results = load_json(RESULTS_FILE)
print(f"Loaded {len(results):,} sentiment results")

# Quick coverage check
has_labor = sum(1 for r in results if r.get('labor_sentiment') is not None)
has_railroad = sum(1 for r in results if r.get('railroad_sentiment') is not None)
has_both = sum(1 for r in results if r.get('labor_sentiment') is not None and r.get('railroad_sentiment') is not None)
print(f"  With labor sentiment:    {has_labor:,}")
print(f"  With railroad sentiment: {has_railroad:,}")
print(f"  With both sentiments:    {has_both:,}")

## Create article_sentiment Table (Dual-Axis Schema)

In [ ]:
con = sqlite3.connect(DB_PATH)
cur = con.cursor()

cur.executescript("""
    DROP TABLE IF EXISTS article_sentiment;

    CREATE TABLE article_sentiment (
        article_id TEXT PRIMARY KEY,
        lccn TEXT NOT NULL,
        issn TEXT,
        year INTEGER NOT NULL,
        category TEXT NOT NULL,
        labor_sentiment TEXT,
        labor_confidence REAL,
        railroad_sentiment TEXT,
        railroad_confidence REAL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );

    CREATE INDEX idx_sentiment_issn_year ON article_sentiment(issn, year);
    CREATE INDEX idx_sentiment_category ON article_sentiment(category);
    CREATE INDEX idx_sentiment_labor ON article_sentiment(labor_sentiment);
    CREATE INDEX idx_sentiment_railroad ON article_sentiment(railroad_sentiment);
    CREATE INDEX idx_sentiment_year ON article_sentiment(year);
""")
con.commit()

print("Table and indexes created.")

In [ ]:
# Insert results in batches
BATCH_SIZE = 5000

for i in tqdm(range(0, len(results), BATCH_SIZE), desc="Inserting rows"):
    batch = results[i:i+BATCH_SIZE]
    cur.executemany("""
        INSERT OR REPLACE INTO article_sentiment
        (article_id, lccn, issn, year, category,
         labor_sentiment, labor_confidence,
         railroad_sentiment, railroad_confidence)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, [
        (r['article_id'], r['lccn'], r['issn'], r['year'], r['category'],
         r.get('labor_sentiment'), r.get('labor_confidence'),
         r.get('railroad_sentiment'), r.get('railroad_confidence'))
        for r in batch
    ])

con.commit()

cur.execute("SELECT COUNT(*) FROM article_sentiment")
count = cur.fetchone()[0]
print(f"\narticle_sentiment table: {count:,} rows")

In [ ]:
# Verification queries

print("=== Labor Sentiment Distribution ===")
cur.execute("""
    SELECT labor_sentiment, COUNT(*) as cnt
    FROM article_sentiment
    WHERE labor_sentiment IS NOT NULL
    GROUP BY labor_sentiment ORDER BY cnt DESC
""")
for row in cur.fetchall():
    print(f"  {row[0]}: {row[1]:,}")

print("\n=== Railroad Sentiment Distribution ===")
cur.execute("""
    SELECT railroad_sentiment, COUNT(*) as cnt
    FROM article_sentiment
    WHERE railroad_sentiment IS NOT NULL
    GROUP BY railroad_sentiment ORDER BY cnt DESC
""")
for row in cur.fetchall():
    print(f"  {row[0]}: {row[1]:,}")

print("\n=== Coverage by Category ===")
cur.execute("""
    SELECT
        category,
        COUNT(*) as total,
        SUM(CASE WHEN labor_sentiment IS NOT NULL THEN 1 ELSE 0 END) as has_labor,
        SUM(CASE WHEN railroad_sentiment IS NOT NULL THEN 1 ELSE 0 END) as has_railroad,
        SUM(CASE WHEN labor_sentiment IS NOT NULL AND railroad_sentiment IS NOT NULL THEN 1 ELSE 0 END) as has_both
    FROM article_sentiment
    GROUP BY category
""")
print(f"  {'category':>10} | {'total':>8} | {'has_labor':>10} | {'has_rr':>8} | {'has_both':>10}")
for row in cur.fetchall():
    print(f"  {row[0]:>10} | {row[1]:>8,} | {row[2]:>10,} | {row[3]:>8,} | {row[4]:>10,}")

print("\n=== 1877 Great Railroad Strike ===")
cur.execute("""
    SELECT labor_sentiment, COUNT(*) FROM article_sentiment
    WHERE year = 1877 AND labor_sentiment IS NOT NULL
    GROUP BY labor_sentiment ORDER BY COUNT(*) DESC
""")
print("  Labor stance:", {r[0]: r[1] for r in cur.fetchall()})
cur.execute("""
    SELECT railroad_sentiment, COUNT(*) FROM article_sentiment
    WHERE year = 1877 AND railroad_sentiment IS NOT NULL
    GROUP BY railroad_sentiment ORDER BY COUNT(*) DESC
""")
print("  Railroad outlook:", {r[0]: r[1] for r in cur.fetchall()})

print("\n=== 1886 Haymarket Affair ===")
cur.execute("""
    SELECT labor_sentiment, COUNT(*) FROM article_sentiment
    WHERE year = 1886 AND labor_sentiment IS NOT NULL
    GROUP BY labor_sentiment ORDER BY COUNT(*) DESC
""")
print("  Labor stance:", {r[0]: r[1] for r in cur.fetchall()})

## Calculate Labor Sentiment Ratios

Aggregate labor stance by newspaper-year. Only articles with `labor_sentiment IS NOT NULL`.

Key DiD variables:
- `anti_labor_intensity` = anti_labor / total (anti-labor stance intensity)
- `labor_biased_ratio` = (pro_labor + anti_labor) / total (editorial vs factual)

In [ ]:
cur.execute("""
    SELECT
        issn,
        year,
        SUM(CASE WHEN labor_sentiment = 'pro_labor' THEN 1 ELSE 0 END) as pro_labor_count,
        SUM(CASE WHEN labor_sentiment = 'anti_labor' THEN 1 ELSE 0 END) as anti_labor_count,
        SUM(CASE WHEN labor_sentiment = 'neutral' THEN 1 ELSE 0 END) as neutral_count,
        COUNT(*) as total_count,
        AVG(labor_confidence) as avg_confidence
    FROM article_sentiment
    WHERE labor_sentiment IS NOT NULL
      AND issn IS NOT NULL AND issn != ''
    GROUP BY issn, year
""")

labor_rows = cur.fetchall()
labor_cols = ['issn', 'year', 'pro_labor_count', 'anti_labor_count',
              'neutral_count', 'total_count', 'avg_confidence']
df_labor = pd.DataFrame(labor_rows, columns=labor_cols)

print(f"Labor aggregated rows (issn-year): {len(df_labor):,}")
print(f"Unique newspapers: {df_labor['issn'].nunique()}")
print(f"Year range: {df_labor['year'].min()}-{df_labor['year'].max()}")

In [ ]:
MIN_ARTICLES = 5

df_labor_ratios = df_labor[df_labor['total_count'] >= MIN_ARTICLES].copy()
df_labor_ratios['pro_labor_ratio'] = df_labor_ratios['pro_labor_count'] / df_labor_ratios['total_count']
df_labor_ratios['anti_labor_ratio'] = df_labor_ratios['anti_labor_count'] / df_labor_ratios['total_count']
df_labor_ratios['neutral_ratio'] = df_labor_ratios['neutral_count'] / df_labor_ratios['total_count']
df_labor_ratios['anti_labor_intensity'] = df_labor_ratios['anti_labor_ratio']
df_labor_ratios['labor_biased_ratio'] = (
    (df_labor_ratios['pro_labor_count'] + df_labor_ratios['anti_labor_count'])
    / df_labor_ratios['total_count']
)
df_labor_ratios = df_labor_ratios.round(4)

print(f"Rows with >= {MIN_ARTICLES} labor articles: {len(df_labor_ratios):,}")
print(f"\n=== Anti-Labor Intensity (key DiD variable) ===")
print(df_labor_ratios['anti_labor_intensity'].describe().round(3))

df_labor_ratios.to_csv(LABOR_RATIOS_OUTPUT, index=False)
print(f"\nSaved to: {LABOR_RATIOS_OUTPUT}")
print(f"Columns: {list(df_labor_ratios.columns)}")

## Calculate Railroad Sentiment Ratios

Aggregate railroad outlook by newspaper-year. Only articles with `railroad_sentiment IS NOT NULL`.

Key DiD variables:
- `railroad_pessimism` = pessimistic / total
- `railroad_biased_ratio` = (optimistic + pessimistic) / total

In [ ]:
cur.execute("""
    SELECT
        issn,
        year,
        SUM(CASE WHEN railroad_sentiment = 'optimistic' THEN 1 ELSE 0 END) as optimistic_count,
        SUM(CASE WHEN railroad_sentiment = 'pessimistic' THEN 1 ELSE 0 END) as pessimistic_count,
        SUM(CASE WHEN railroad_sentiment = 'neutral' THEN 1 ELSE 0 END) as neutral_count,
        COUNT(*) as total_count,
        AVG(railroad_confidence) as avg_confidence
    FROM article_sentiment
    WHERE railroad_sentiment IS NOT NULL
      AND issn IS NOT NULL AND issn != ''
    GROUP BY issn, year
""")

rr_rows = cur.fetchall()
rr_cols = ['issn', 'year', 'optimistic_count', 'pessimistic_count',
           'neutral_count', 'total_count', 'avg_confidence']
df_rr = pd.DataFrame(rr_rows, columns=rr_cols)

print(f"Railroad aggregated rows (issn-year): {len(df_rr):,}")
print(f"Unique newspapers: {df_rr['issn'].nunique()}")
print(f"Year range: {df_rr['year'].min()}-{df_rr['year'].max()}")

In [ ]:
df_rr_ratios = df_rr[df_rr['total_count'] >= MIN_ARTICLES].copy()
df_rr_ratios['optimistic_ratio'] = df_rr_ratios['optimistic_count'] / df_rr_ratios['total_count']
df_rr_ratios['pessimistic_ratio'] = df_rr_ratios['pessimistic_count'] / df_rr_ratios['total_count']
df_rr_ratios['neutral_ratio'] = df_rr_ratios['neutral_count'] / df_rr_ratios['total_count']
df_rr_ratios['railroad_pessimism'] = df_rr_ratios['pessimistic_ratio']
df_rr_ratios['railroad_biased_ratio'] = (
    (df_rr_ratios['optimistic_count'] + df_rr_ratios['pessimistic_count'])
    / df_rr_ratios['total_count']
)
df_rr_ratios = df_rr_ratios.round(4)

print(f"Rows with >= {MIN_ARTICLES} railroad articles: {len(df_rr_ratios):,}")
print(f"\n=== Railroad Pessimism (key DiD variable) ===")
print(df_rr_ratios['railroad_pessimism'].describe().round(3))

df_rr_ratios.to_csv(RAILROAD_RATIOS_OUTPUT, index=False)
print(f"\nSaved to: {RAILROAD_RATIOS_OUTPUT}")
print(f"Columns: {list(df_rr_ratios.columns)}")

## Yearly Summary

In [ ]:
print("=== Labor: Yearly Summary ===")
labor_yearly = df_labor_ratios.groupby('year').agg({
    'anti_labor_intensity': 'mean',
    'labor_biased_ratio': 'mean',
    'total_count': 'sum',
    'issn': 'nunique',
}).rename(columns={'issn': 'newspaper_count'}).round(3)
print(labor_yearly.to_string())

print("\n=== Railroad: Yearly Summary ===")
rr_yearly = df_rr_ratios.groupby('year').agg({
    'railroad_pessimism': 'mean',
    'railroad_biased_ratio': 'mean',
    'total_count': 'sum',
    'issn': 'nunique',
}).rename(columns={'issn': 'newspaper_count'}).round(3)
print(rr_yearly.to_string())

In [ ]:
con.close()
print("\nDone. Database connection closed.")
print(f"\nOutputs:")
print(f"  1. article_sentiment table in {DB_PATH}")
print(f"     - labor_sentiment: pro_labor / anti_labor / neutral (NULL for railroad-only)")
print(f"     - railroad_sentiment: optimistic / pessimistic / neutral (NULL for labor-only)")
print(f"     - 'both' articles have BOTH columns filled")
print(f"  2. {LABOR_RATIOS_OUTPUT}")
print(f"     Key variable: anti_labor_intensity (anti-labor / total, per newspaper-year)")
print(f"  3. {RAILROAD_RATIOS_OUTPUT}")
print(f"     Key variable: railroad_pessimism (pessimistic / total, per newspaper-year)")
print(f"\nNext steps:")
print(f"  - Join ratio CSVs with master.csv on issn + year for full panel DiD analysis")